In [ ]:
from matplotlib import animation
import numpy as np
import sympy as sp
import matplotlib
import random
import matplotlib.pyplot as plt
from matplotlib import animation, rc, patches
from IPython.display import HTML

from matplotlib import animation, rc, patches
from IPython.display import HTML

L = 4.0
S = 5.0
A0 = 0.1
k = 2.0
qn = 5


In [ ]:

def N1(x, x1e, x2e):
    le = x2e - x1e
    return (x2e - x) / le

def N2(x, x1e, x2e):
    le = x2e - x1e
    return (x - x1e) / le

def dN1dx(x, xle, x2e):
    le = x2e - xle
    return -1. / le

def dN2dx(x, x1e, x2e):
    le = x2e - x1e
    return 1. / le

gauss_2 = np.array([[-1 / np.sqrt(3.), 1.], [1 / np.sqrt(3.), 1.]])


In [ ]:

n_ele = 16
n_node = n_ele + 1
node_X = np.linspace(0, L, n_node)

elements = np.zeros((n_ele, 2), dtype=int)
elements[:, 0] = np.arange(n_ele)
elements[:, 1] = np.arange(n_ele) + 1

T_sols = []

T_0 = np.zeros((n_node))
T_sols.append(T_0)

delta_t = 0.1
total_steps = 2000

for j in range(total_steps):
    K = np.zeros((n_node, n_node))
    M = np.zeros((n_node, n_node))
    F_s = np.zeros((n_node))
    F_q = np.zeros((n_node))
    F_0 = np.zeros((n_node))

    for i in range(n_ele):
        x1e = node_X[elements[i, 0]]
        x2e = node_X[elements[i, 1]]
        le = x2e - x1e

        T1e = T_0[elements[i, 0]]
        T2e = T_0[elements[i, 1]]

        Ke = A0 * k / le * np.array([[1., -1], [-1., 1]])

        K[elements[i, 0], elements[i, 0]] += Ke[0, 0]
        K[elements[i, 0], elements[i, 1]] += Ke[0, 1]
        K[elements[i, 1], elements[i, 0]] += Ke[1, 0]
        K[elements[i, 1], elements[i, 1]] += Ke[1, 1]

        Fes = np.zeros((2))
        for ip in range(len(gauss_2)):
            x_i = 0.5 * (x1e + x2e) + 0.5 * gauss_2[ip, 0] * le
            J = le / 2.
            Nvec = np.array([N1(x_i, x1e, x2e), N2(x_i, x1e, x2e)])
            Fes += J * gauss_2[ip, 1] * Nvec * S

        F_s[elements[i, 0]] += Fes[0]
        F_s[elements[i, 1]] += Fes[1]

        Me = np.zeros((2, 2))
        for ip in range(len(gauss_2)):
            x_i = 0.5 * (x1e + x2e) + 0.5 * gauss_2[ip, 0] * le
            J = le / 2.
            Nvec = np.array([N1(x_i, x1e, x2e), N2(x_i, x1e, x2e)])
            Me += J * gauss_2[ip, 1] * np.outer(Nvec, Nvec)

        M[elements[i, 0], elements[i, 0]] += Me[0, 0]
        M[elements[i, 0], elements[i, 1]] += Me[0, 1]
        M[elements[i, 1], elements[i, 0]] += Me[1, 0]
        M[elements[i, 1], elements[i, 1]] += Me[1, 1]

        fe0 = np.zeros((2))
        for ip in range(len(gauss_2)):
            x_i = 0.5 * (x1e + x2e) + 0.5 * gauss_2[ip, 0] * le
            J = le / 2.
            Nvec = np.array([N1(x_i, x1e, x2e), N2(x_i, x1e, x2e)])
            T_i = N1(x_i, x1e, x2e) * T1e + N2(x_i, x1e, x2e) * T2e
            fe0 += J * gauss_2[ip, 1] * Nvec * T_i

        F_0[elements[i, 0]] += fe0[0]
        F_0[elements[i, 1]] += fe0[1]

    # Solve problem for temp
    n_E = 1  # nodes 1 essential boundary conditions
    n_F = n_node-1  # the rest of the nodes are free
    # External fluxes, only last node has flux
    F_q = np.zeros((n_node))
    F_q[-1] = qn*A0  # only last node has flux
    # known temperature
    d_E = np.array([0.])  # node 1 is fixed temperature T=0
    # partition of stiffness matrix
    K_E = K[0:n_E, 0:n_E]
    K_F = K[n_E:, n_E:]
    K_EF = K[0:n_E, n_E:]
    # partition of mass matrix
    M_E = M[0:n_E, 0:n_E]
    M_F = M[n_E:, n_E:]
    M_EF = M[0:n_E, n_E:]
    # known boundary flux and source term and previous time step 'force'
    F_F = - F_q[n_E:] + F_s[n_E:]+(1./delta_t)*F_0[n_E:]
    # solving A\b
    d_F = np.linalg.solve((1.0/delta_t)*M_F+K_F, F_F -
                            np.dot((1.0/delta_t)*M_EF.transpose()+K_EF.transpose(), d_E))
    # you can print if you want
    # print(d_F)
    # Assemble the vector with the full solution [d_E,d_F], becase right now I have two
    # separate vectors
    node_T = np.zeros((n_node))
    node_T[0:n_E] = d_E
    node_T[n_E:] = d_F
    # store into the array of solutions
    T_sols.append(node_T)
    # reset time step
    T_0 = node_T



In [ ]:
def T_exact(x):
    return -12.5*x ** 2 + 97.5*x


def dTdx_exact(x):
    return -25*x + 97.5



# Plot function over each domain

fig, (ax1) = plt.subplots(1, 1, figsize=(11, 5))
frac = 200
for j in range(int(total_steps/frac)):
    node_T = T_sols[j*frac]
    # Plot the FE solution
    # Loop over elements
    for i in range(n_ele):
        # Read the nodes that make up this element 'i'
        n1 = elements[i, 0]
        n2 = elements[i, 1]
        # Beginning and end 'x' coordinates of this element
        xle = node_X[n1]
        x2e = node_X[n2]
        # Get the nodal values of the function 'T'
        Tle = node_T[n1]
        T2e = node_T[n2]
        # Eval the shape function evaluation for multiple x in this element
        x_vec = np.linspace(xle, x2e, 50)
        # use the shape functions to interpolate from the nodal T values to all the 'x'
        T_vec = N1(x_vec, xle, x2e)*Tle + N2(x_vec, xle, x2e)*T2e
        # gradient is going to be constant in this case
        dTdx = dN1dx(x_vec, xle, x2e)*Tle + dN2dx(x_vec, xle, x2e)*T2e
        dTdx_vec = dTdx*np.ones(50)
        # plot function T
        # some trick to get the legend
        # some trick to get the legend
        if i == 0:
            ax1.plot(x_vec, T_vec, color=[
                     0.5, j/total_steps*frac, j/total_steps*frac], label='time=%f' %        (j*frac*delta_t))
        else:
            ax1.plot(x_vec, T_vec, color=[
                     0.5, j/total_steps*frac, j/total_steps*frac])


# Plot the exact solution
x_ls = np.linspace(0, 4, 100)
ax1.plot(x_ls, T_exact(x_ls), '-', zorder=4, color='red', label='exact')   
ax1.grid(True)
ax1.set_title('T')
ax1.set(xlabel='x [m]', ylabel='T [C]')
ax1.legend(loc=2)
plt.show()

In [ ]:

time_steps = range(len(T_sols))
node_temperatures = np.array(T_sols)

num_time_steps_to_plot = 10

fig, ax = plt.subplots()

for time_step in range(0, len(time_steps), len(time_steps) // num_time_steps_to_plot):
    x_positions = np.linspace(0, L, n_node)
    ax.plot(x_positions, node_temperatures[time_step], label=f'Time Step {time_step}')

ax.set_xlabel('Position (m)')
ax.set_ylabel('Temperature (°C)')
ax.set_title(f'Temperature Distribution at Selected Time Steps')
ax.legend(loc='upper right')
plt.show()


In [ ]:

fig = plt.figure(figsize=(5, 5))
ax = plt.subplot(111, frameon=True)
ax.set_xlim((0, 4))
ax.set_ylim((0, 200))
x_vec = np.linspace(0, 4, n_node)
P, = ax.plot(x_vec, T_sols[0])

dt = 100
frames = len(T_sols)

def update(frame):
    P.set_data(x_vec, T_sols[frame])

anim = animation.FuncAnimation(fig, update, frames=frames, interval=dt)
plt.show()